## Pandas 학습

### 데이터 통합

#### merge
- 옆으로 통합(합치기)
- DataBase의 JOIN과 동일한 기능

In [22]:
import pandas as pd

In [40]:
# header 엑셀에서 필요없는 제목줄 등을 삭제
# skupfooter 아래 쪽에서 필요없는 행들을 삭제
df_left = pd.read_excel('./data/sample_1.xlsx',header=1,skipfooter=2,usecols='a:d')
df_left

,국적코드,성별,입국객수,전년동기
0,A01,남성,106320,85815
1,A01,여성,191436,125241
2,A31,남성,319,299
3,A31,여성,42,54
4,A18,남성,158912,124486
5,A18,여성,232943,163466


In [ ]:
df_right = pd.read_excel('./data/sample_codemaster.xlsx',usecols='a:b')
df_right

In [33]:
df_total = pd.merge(left=df_left,right=df_right,how='left',
            left_on='국적코드',right_on='국적코드')

In [65]:
# NaN : 해당 국적코드에 대한 국가명이 명시되어있지 않음
df_total

,국적코드,성별,입국객수,전년동기,국적명
0,A01,남성,106320,85815,일본
1,A01,여성,191436,125241,일본
2,A31,남성,319,299,NaN
3,A31,여성,42,54,NaN
4,A18,남성,158912,124486,중국
5,A18,여성,232943,163466,중국


In [32]:
## DB INNER JOIN과 동일
df_total_inner = pd.merge(left=df_left,right=df_right,how='inner',
            left_on='국적코드',right_on='국적코드')



### append
- 데이터를 합칠 때 데이터 수가 늘도록 합치는 방법(아래로 통합)
- DB의 UNION과 동일한 기능

In [50]:
## append를 하기 위한 데이터 하나 더 생성
df_left2 = pd.read_excel('./data/sample_2.xlsx',header=1,skipfooter=2,usecols='a:d')
df_left2

,국적코드,성별,입국객수,전년동기
0,A01,남성,92556,75992
1,A01,여성,163737,115397
2,A18,남성,155540,129908
3,A18,여성,249023,191236


In [59]:
# df_all = df_total + df_total2
# 중요! append()가 사라졌으므로 pd.concat()으로 사용할 것
pd.concat([df_left,df_left2])

,국적코드,성별,입국객수,전년동기
0,A01,남성,106320,85815
1,A01,여성,191436,125241
2,A31,남성,319,299
3,A31,여성,42,54
4,A18,남성,158912,124486
5,A18,여성,232943,163466
0,A01,남성,92556,75992
1,A01,여성,163737,115397
2,A18,남성,155540,129908
3,A18,여성,249023,191236


In [61]:
pd.concat([df_total,df_left2]) #이렇게 합치면 안된다.

,국적코드,성별,입국객수,전년동기,국적명
0,A01,남성,106320,85815,일본
1,A01,여성,191436,125241,일본
2,A31,남성,319,299,NaN
3,A31,여성,42,54,NaN
4,A18,남성,158912,124486,중국
5,A18,여성,232943,163466,중국
0,A01,남성,92556,75992,NaN
1,A01,여성,163737,115397,NaN
2,A18,남성,155540,129908,NaN
3,A18,여성,249023,191236,NaN


In [63]:
# df_left2를 가지고 df_total2를 만듦. 국적명을 제대로 받기 위해
df_total2 = pd.merge(left=df_left2,right=df_right,how='left',left_on='국적코드',right_on='국적코드')
df_total2

,국적코드,성별,입국객수,전년동기,국적명
0,A01,남성,92556,75992,일본
1,A01,여성,163737,115397,일본
2,A18,남성,155540,129908,중국
3,A18,여성,249023,191236,중국


In [67]:
# ignore_index 는 DataFrame의 자체 인덱스를 무시하고 새로 인덱스를 생성
df_all = pd.concat([df_total,df_total2],ignore_index='True')

In [75]:
# DF 저장 .to_***
# index = False DataFrame이 만드는 임의의 인덱스는 저장 안 됨
df_all.to_excel('./data/df_all.xlsx',index=False)

In [79]:
df_total['기준년월'] = '2019-11'
df_total

,국적코드,성별,입국객수,전년동기,국적명,기준년월
0,A01,남성,106320,85815,일본,2019-11
1,A01,여성,191436,125241,일본,2019-11
2,A31,남성,319,299,NaN,2019-11
3,A31,여성,42,54,NaN,2019-11
4,A18,남성,158912,124486,중국,2019-11
5,A18,여성,232943,163466,중국,2019-11


In [80]:
df_total2 ['기준년월'] = '2019-12'
df_total2

,국적코드,성별,입국객수,전년동기,국적명,기준년월
0,A01,남성,92556,75992,일본,2019-12
1,A01,여성,163737,115397,일본,2019-12
2,A18,남성,155540,129908,중국,2019-12
3,A18,여성,249023,191236,중국,2019-12


In [110]:
df_all = pd.concat([df_total,df_total2],ignore_index=True)

df_all.to_excel('./data/df_all.xlsx',index=False)


#### Pivot 테이블

통계를 내면서 테이블을 세로로된 DataFrame을 가로로 바꿔주는 기능

DB에서 피벗작업은 힘듦, 파이썬에서 피벗하는 게 쉬움

In [112]:
pivot = df_all.pivot_table(values='입국객수',index='국적코드',
                   columns='기준년월',aggfunc=f'{"sum"}')

#aggfuc 옵션
# mean 평균
# sum 합계
# min 최솟값
# median 중앙값
# max 최댓값
# count 개수
# nunique 중복을 제거한 후 개수(원소 개수)

In [115]:
pivot.style.highlight_min(color='green')

기준년월,2019-11,2019-12
국적코드,,
A01,297756.000000,256293.000000
A18,391855.000000,404563.000000
A31,361.000000,nan
